In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'data1/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size=28
num_labels=10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1,image_size * image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset,labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [14]:
train_subset = 10000
graph = tf.Graph()

In [22]:
with graph.as_default():
    
    #tf constant
    tf_train_dataset = tf.constant(train_dataset[:train_subset,:])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #tf variable
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros(num_labels))
    
    # Training computation
    
    logits = tf.matmul(tf_train_dataset , weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits)
    )
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    #These are not part of the training , but can help us in judging the accuracy figures
    train_prediction = tf.nn.softmax(logits)
    valid_prediction  = tf.nn.softmax(tf.matmul(valid_dataset, weights)+biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_dataset,weights) + biases)
    
    

In [23]:
num_steps = 801
def accuracy(predictions,labels):
    return 100* (np.sum(np.argmax(predictions,1)== np.argmax(labels,1)))/ predictions.shape[0]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Hyper params initialized..')
    for step in range(num_steps):
        _,l,predictions = session.run([optimizer,loss,train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Hyper params initialized..
Loss at step 0: 18.056057
Training accuracy: 7.0%
Validation accuracy: 10.0%
Loss at step 100: 2.286180
Training accuracy: 71.0%
Validation accuracy: 71.0%
Loss at step 200: 1.828773
Training accuracy: 75.0%
Validation accuracy: 73.0%
Loss at step 300: 1.590837
Training accuracy: 76.0%
Validation accuracy: 74.0%
Loss at step 400: 1.430886
Training accuracy: 77.0%
Validation accuracy: 75.0%
Loss at step 500: 1.312023
Training accuracy: 78.0%
Validation accuracy: 75.0%
Loss at step 600: 1.218273
Training accuracy: 79.0%
Validation accuracy: 75.0%
Loss at step 700: 1.141401
Training accuracy: 79.0%
Validation accuracy: 75.0%
Loss at step 800: 1.076732
Training accuracy: 80.0%
Validation accuracy: 76.0%
Test accuracy: 83.0%


In [24]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [25]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.011055
Minibatch accuracy: 5.0%
Validation accuracy: 10.0%
Minibatch loss at step 500: 1.382227
Minibatch accuracy: 76.0%
Validation accuracy: 77.0%
Minibatch loss at step 1000: 1.132931
Minibatch accuracy: 78.0%
Validation accuracy: 77.0%
Minibatch loss at step 1500: 0.972798
Minibatch accuracy: 81.0%
Validation accuracy: 78.0%
Minibatch loss at step 2000: 1.125831
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step 2500: 0.766437
Minibatch accuracy: 82.0%
Validation accuracy: 79.0%
Minibatch loss at step 3000: 1.118462
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Test accuracy: 86.0%


In [7]:
batch_size=128
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(np.float32,shape=(batch_size,image_size * image_size))
    tf_train_labels = tf.placeholder(np.float32,shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables & Layer 1
    
    weight_layer_inp = tf.Variable(tf.truncated_normal([image_size*image_size , 80]))
    bias_layer_inp = tf.Variable(tf.zeros(80))
    logits_relu_layer_inp = tf.nn.relu(tf.matmul(tf_train_dataset,weight_layer_inp)+bias_layer_inp)
    
    #  Layer 2 - Final Layer
    
    weights = tf.Variable(tf.truncated_normal([80,num_labels]))
    biases = tf.Variable(tf.zeros(num_labels))
    logits = tf.matmul(logits_relu_layer_inp,weights) + biases
    
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,weight_layer_inp)+bias_layer_inp), weights) + biases)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,weight_layer_inp)+bias_layer_inp), weights) + biases)

In [10]:
num_steps = 30000

def accuracy(predictions,labels):
    return 100* (np.sum(np.argmax(predictions,1)== np.argmax(labels,1)))/ predictions.shape[0]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 98.286636
Minibatch accuracy: 10.0%
Validation accuracy: 26.0%
Minibatch loss at step 500: 1.607130
Minibatch accuracy: 75.0%
Validation accuracy: 72.0%
Minibatch loss at step 1000: 0.897938
Minibatch accuracy: 82.0%
Validation accuracy: 75.0%
Minibatch loss at step 1500: 0.893911
Minibatch accuracy: 80.0%
Validation accuracy: 79.0%
Minibatch loss at step 2000: 1.112662
Minibatch accuracy: 71.0%
Validation accuracy: 78.0%
Minibatch loss at step 2500: 0.666939
Minibatch accuracy: 83.0%
Validation accuracy: 80.0%
Minibatch loss at step 3000: 1.262158
Minibatch accuracy: 75.0%
Validation accuracy: 78.0%
Minibatch loss at step 3500: 0.547273
Minibatch accuracy: 87.0%
Validation accuracy: 81.0%
Minibatch loss at step 4000: 0.857965
Minibatch accuracy: 78.0%
Validation accuracy: 80.0%
Minibatch loss at step 4500: 0.546081
Minibatch accuracy: 85.0%
Validation accuracy: 81.0%
Minibatch loss at step 5000: 1.643272
Minibatch accuracy: 71.0%
Validation accura